In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
appName("Sneha Spark Session").\
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [5]:
! hadoop fs -cat /public/trendytech/datasets/windowdata.csv |head

country,weeknum,numinvoices,totalquantity,invoicevalue
Spain,49,1,67,174.72
Germany,48,11,1795,3309.75
Lithuania,48,3,622,1598.06
Germany,49,12,1852,4521.39
Bahrain,51,1,54,205.74
Iceland,49,1,319,711.79
India,51,5,95,276.84
Australia,50,2,133,387.95
Italy,49,1,-2,-17.0


In [6]:
order_df = spark.read.\
format('csv').\
option("inferSchema", True).\
option("header", True).\
load('/public/trendytech/datasets/windowdata.csv')

In [7]:
order_df.sort("country","weeknum").show()

+---------------+-------+-----------+-------------+------------+
|        country|weeknum|numinvoices|totalquantity|invoicevalue|
+---------------+-------+-----------+-------------+------------+
|      Australia|     48|          1|          107|      358.25|
|      Australia|     49|          1|          214|       258.9|
|      Australia|     50|          2|          133|      387.95|
|        Austria|     50|          2|            3|      257.04|
|        Bahrain|     51|          1|           54|      205.74|
|        Belgium|     48|          1|          528|       346.1|
|        Belgium|     50|          2|          285|      625.16|
|        Belgium|     51|          2|          942|      838.65|
|Channel Islands|     49|          1|           80|      363.53|
|         Cyprus|     50|          1|          917|     1590.82|
|        Denmark|     49|          1|          454|      1281.5|
|        Finland|     50|          1|         1254|       892.8|
|         France|     48|

In [8]:
from pyspark.sql import *

In [13]:
from pyspark.sql.functions import *

In [14]:
mywindow = Window.partitionBy("country").orderBy("weeknum").rowsBetween(Window.unboundedPreceding,Window.currentRow)

In [15]:
result = order_df.withColumn("running_total",sum("invoicevalue").over(mywindow))

In [16]:
result.show()

+-------+-------+-----------+-------------+------------+------------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|     running_total|
+-------+-------+-----------+-------------+------------+------------------+
| Sweden|     50|          3|         3714|      2646.3|            2646.3|
|Germany|     48|         11|         1795|     3309.75|           3309.75|
|Germany|     49|         12|         1852|     4521.39|           7831.14|
|Germany|     50|         15|         1973|     5065.79|          12896.93|
|Germany|     51|          5|         1103|     1665.91|          14562.84|
| France|     48|          4|         1299|     2808.16|           2808.16|
| France|     49|          9|         2303|     4527.01|           7335.17|
| France|     50|          6|          529|      537.32|           7872.49|
| France|     51|          5|          847|     1702.87|           9575.36|
|Belgium|     48|          1|          528|       346.1|             346.1|
|Belgium|   